<a href="https://colab.research.google.com/github/yscope75/CS2225.CH2001020/blob/master/Image_captioning_Master_courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.datasets as dset
import torchvision.datasets.utils as dset_utils
import torchvision.transforms as transforms
import os

In [2]:
annotation_folder = os.path.join(os.path.abspath('.') + '/coco/annotations/')
image_folder = os.path.join(os.path.abspath('.') + '/coco/train2014/')
# Download and unzip annotations 
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                        download_root=annotation_folder,
                                        extract_root=annotation_folder,
                                        filename='captions.zip')
# Download and unzion images
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/zips/train2014.zip',
                                        download_root=image_folder,
                                        extract_root=image_folder,
                                        filename='train2014.zip')

Extracting /content/coco/annotations/captions.zip to /content/coco/annotations/


Extracting /content/coco/train2014/train2014.zip to /content/coco/train2014/


In [ ]:
!ls coco/annotations

captions_train2014.json   instances_val2014.json
captions_val2014.json	  person_keypoints_train2014.json
instances_train2014.json  person_keypoints_val2014.json


In [ ]:
cap = dset.CocoCaptions(root = './coco/images', 
                        annFile = './coco/annotations/captions_train2014.json',
                        transform=transforms.ToTensor())



loading annotations into memory...


FileNotFoundError: ignored

In [ ]:
!ls

coco  sample_data


In [ ]:
!ls ./coco

annotation  images
